  <center>

<a href="https://github.com/QuantumBFS/Yao.jl"><img alt="幺" height="200" src="https://quantumbfs.github.io/Yao.jl/latest/assets/logo.svg" style="margin: 0; border: none; box-shadow: none; background: none;" width="200"></a>

<h3>EXTENSIBLE EFFICIENT QUANTUM ALGORITHM DESIGN FOR HUMANS.</h3>


</center>

## Variational Quantum Algorithm

- Variational Quantum Eigensolver
- Quantum Circuit Born Machine
- Quantum Approximate Optimization Algorithm
- Quantum Circuit Learning
- etc.

### Variational Quantum Eigensolver

![](https://riverlane.io/wp-content/uploads/image-2.png)

![](https://riverlane.io/wp-content/uploads/image-3.png)

### Quantum Circuit Born Machine (arxiv: 1804.04168)

![](https://quantumbfs.github.io/Yao.jl/latest/assets/figures/differentiable.png)

### MPS based Generative Models (arxiv 1803.11537)

![](assets/mps-circuit.png)

### A General Scheme for Variational Quantum Algorithms

![](https://qmlt.readthedocs.io/en/latest/_images/variational_detail.png)

### Requirements & Challenge

- **Automatic Differentiation** is required for calculating the gradient of the parameters
- A flexible way of **dispatching parameters** is required.
- **High performance** in simulating **6~20** qubits classically is required.
- **Extensibility on customized feature** is crucial since there is not much analytical guidance yet.

## Reference

- [The full quantum computation stack](https://riverlane.io/news/the-full-quantum-computation-stack/)
- [Documentation: Quantum Machine Learning Toolbox](https://qmlt.readthedocs.io/en/latest/variational.html)
- [Variational Quantum Eigensolver](https://doi.org/10.1038/ncomms5213)
- [Quantum Circuit Born Machine](https://arxiv.org/abs/1804.04168)
- [Towards Quantum Machine Learning with Tensor Networks](https://arxiv.org/pdf/1803.11537.pdf)

## Our solution: Yao.jl

### Automatic Differentiation & Parameter Dispatch

In [ ]:
using Yao

circuit = chain(4, repeat(4, H, 1:4), put(4, 3=>Rz(0.5)), control(2, 1=>X), put(4, 4=>Ry(0.2)))

Here, we demonstrate our AD with a random state input and a loss of the following operator $op$ as $\langle\psi|op|\psi\rangle$, the derivative will be

$$
\frac{\partial f}{\partial \theta} = 2\mathcal{R}\{\frac{\partial f}{\partial \psi^*}\frac{\partial \psi^*}{\partial \theta}\}
$$

In [ ]:
circuit = circuit |> autodiff(:BP)
op = put(4, 3=>Y)
ψ = rand_state(4)
ψ |> circuit
δ = ψ |> op # 
backward!(δ, circuit)
gradient(circuit)

### Gather & Dispatch Parameters

In [ ]:
parameters(circuit)

In [ ]:
dispatch!(circuit, rand(2))

### High Performance


<div>
<img alt="benchmark-1" style="float: left;" height="600" src="https://quantumbfs.github.io/Yao.jl/stable/assets/benchmarks/xyz-bench.png" style="margin: 0; border: none; box-shadow: none; background: none;" width="400">
<img alt="benchmark-2" style="float: right;" height="600" src="https://quantumbfs.github.io/Yao.jl/stable/assets/benchmarks/repeatxyz-bench.png" style="margin: 0; border: none; box-shadow: none; background: none;" width="400">
<img alt="benchmark-2" style="float: left;"  height="600" src="https://quantumbfs.github.io/Yao.jl/stable/assets/benchmarks/cxyz-bench.png" style="margin: 0; border: none; box-shadow: none; background: none;" width="400">
</div>

#### More complex task

1. [Dynamic Quantum Phase Transition](https://github.com/GiggleLiu/dynamicQPF): **4.3x** faster (comparing to ProjectQ)
2. [Quantum Circuit Born Machine](https://github.com/QuantumBFS/CuYao.jl/issues/1): **62.4x** faster on NVIDIA Titan V on batched input states (comparing to Yao's own CPU)

### High Extensibility: Block Tree System

The following code constructs a Quantum Fourier Transformation circuit from scratch.

![](https://quantumbfs.github.io/Yao.jl/latest/assets/figures/qft.png)

In [ ]:
using Yao

# Control-R(k) gate in block-A
A(i::Int, j::Int, k::Int) = control(i, j=>shift(2π/(1<<k)))
# block-B
B(n::Int, i::Int) = chain(i==j ? kron(i=>H) : A(j, i, j-i+1) for j = i:n)
QFT(n::Int) = chain(n, B(n, i) for i = 1:n)

In [ ]:
QFT(3)

Any circuit defined with Yao.jl is indenpendent from register, etc. You can always re-use what you defined, like a program.

In [ ]:
chain(2, QFT(2), put(1=>H), QFT(2))

Or, you can define your own type for custom blocks just like other Julia types.

In [ ]:
using Yao.Blocks
struct QFTBlock{N} <: PrimitiveBlock{N,ComplexF64} end

get_circuit(::QFTBlock{N}) where N = QFT(N)
Blocks.mat(c::QFTBlock) = mat(get_circuit(c))
# apply!(register, blk::QFTBlock) = # QFT can be simulated efficiently by using FFT

And use it in other circuits like a sub-program

In [ ]:
function PEBlock(UG::GeneralMatrixGate, n_reg::Int, n_b::Int)
    nbit = n_b + n_reg
    # Apply Hadamard Gate.
    hs = repeat(nbit, H, 1:n_reg)

    # Construct a control circuit.
    control_circuit = chain(nbit)
    for i = 1:n_reg
        push!(control_circuit, control(nbit, (i,), (n_reg+1:nbit...,)=>UG))
        if i != n_reg
            UG = matrixgate(mat(UG) * mat(UG))
        end
    end

    # Inverse QFT Block.
    iqft = concentrate(nbit, QFTBlock{n_reg}() |> adjoint,[1:n_reg...,])
    chain(hs, control_circuit, iqft)
end

We now have a new kind of block inside the circuit: **QFTBlock**

In [ ]:
PEBlock(matrixgate(rand(2, 2)), 3, 1)

### Towards High Level Quantum Programming

<div>
    <img alt="block tree" style="float: left;" height="600" src="assets/block-tree.png" style="margin: 0; border: none; box-shadow: none; background: none;" width="400">
<img alt="benchmark-1" style="float: right;" height="600"      src="https://www.researchgate.net/profile/Peter_Fritzson/publication/228792639/figure/fig1/AS:393782852898820@1470896556105/Abstract-syntax-tree-of-the-while-loop_W640.jpg" style="margin: 0; border: none; box-shadow: none; background: none;" width="400">
</div>

### Heterogeneous Computing

Our framework is natural on different hardware by making use of Julia's **type system** and **multiple dispatch**.


```julia
for MT in [:SDDiagonal, :SDPermMatrix, :AbstractMatrix, :IMatrix, :SDSparseMatrixCSC]
@eval function u1apply!(state::CuVecOrMat, U1::$MT, ibit::Int)
    kf = u1_kernel(U1, ibit::Int)
    X, Y = cudiv(size(state)...)
    @cuda threads=X blocks=Y simple_kernel(kf, state)
    state
end
end
```

#### Pure Julia Implementation

Our framework is implemented in **pure Julia**, there is **no second language** involved, any part the framework is **easily readable and accessible**.

### Demo: [Quantum Circuit Born Machine](https://quantumbfs.github.io/Yao.jl/latest/tutorial/QCBM/)

## On-going Projects & Outlook

<center>
![GSoC](https://3.bp.blogspot.com/-qbcVeQlCj54/W-xm_OlMg5I/AAAAAAAABu8/KCnTxayUm1Y3TlpcQFQbsNK5nA6K5M6pgCLcBGAs/s200/GSoC%2B-%2BVertical%2BNarrow%2B-%2BGray%2BText%2B-%2BWhite%2BBG.png)
</center>
- Noisy Circuit Simulation
- Plotting
- Tensor Network Based Quantum Circuit Simulation

### Outlook

- Quantum circuit optimizer based on block tree data structure
- Compile block tree to real hardware (QASM, Quil), hybrid computation via multiple hardware (quantum, CPU, GPU, TPU)
- Distributed Computing via Julia `DistributedArrays`

Follow us on Github: https://github.com/QuantumBFS